In [219]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('OpenRA_FileCommits.csv')

df.drop('Unnamed: 0', axis=1, inplace=True)

df['fullpath'] = df['new_path']
df.drop('project_name', axis=1, inplace=True)
df.drop('project_path', axis=1, inplace=True)
df.drop('new_path', axis=1, inplace=True)
df.drop('old_path', axis=1, inplace=True)

df.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,filename,parents,fullpath
0,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Blowfish.cs,NaN,MixBrowser\Blowfish.cs
1,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MM.DAT,NaN,MixBrowser\MM.DAT
2,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixBrowser.csproj,NaN,MixBrowser\MixBrowser.csproj
3,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixEntry.cs,NaN,MixBrowser\MixEntry.cs
4,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Program.cs,NaN,MixBrowser\Program.cs


In [220]:
# This list is really rudimentary, but should let us guess if something is a bug or feature
bug_indicators = ['bug', 'fix', 'issue', 'crash', 'error', 'broke', 'break', 'catastrophic', 'report', 'critical', 'urgent', 'unable']

# Let's determine if a commit relates to a bug or a feature
def set_is_bug(row):

    message = row['message'].lower()

    #row['is_bug'] = message
    
    if bool([ele for ele in bug_indicators if(ele in message)]):
        row['is_bug'] = 1
    else:
        row['is_bug'] = 0

    return row

df = df.apply(set_is_bug, axis=1)
df.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,filename,parents,fullpath,is_bug
0,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Blowfish.cs,NaN,MixBrowser\Blowfish.cs,0
1,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MM.DAT,NaN,MixBrowser\MM.DAT,0
2,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixBrowser.csproj,NaN,MixBrowser\MixBrowser.csproj,0
3,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,MixEntry.cs,NaN,MixBrowser\MixEntry.cs,0
4,b59ba43934a3a6837410db51cf60157cf854e52d,openra first commit!\r\n\r\ngit-svn-id: svn://...,chrisf,2007-06-19 08:51:17+00:00,True,False,0,1350,1350,11,bleed,Program.cs,NaN,MixBrowser\Program.cs,0


In [221]:
df_files = pd.read_csv('filesizes.csv')

df_files.drop('Unnamed: 0', axis=1, inplace=True)

def fix_file_path(row):
    if row['path'] == '.':
        row['fullpath'] = row['project'] + '\\' + row['filename']
    else:
        row['fullpath'] = row['project'] + '\\' + row['path'] + '\\' + row['filename']
    return row

df_files = df_files.apply(fix_file_path, axis=1)

df_files.head()

,fullpath,project,path,filename,ext,lines
0,OpenRA.Game\Activities\Activity.cs,OpenRA.Game,Activities,Activity.cs,.cs,291
1,OpenRA.Game\Activities\CallFunc.cs,OpenRA.Game,Activities,CallFunc.cs,.cs,33
2,OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,.cs,645
3,OpenRA.Game\CPos.cs,OpenRA.Game,.,CPos.cs,.cs,150
4,OpenRA.Game\CryptoUtil.cs,OpenRA.Game,.,CryptoUtil.cs,.cs,261


In [222]:
df_merged = pd.merge(df, df_files, left_on='fullpath', right_on='fullpath')

df_merged.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,branches,filename_x,parents,fullpath,is_bug,project,path,filename_y,ext,lines
0,7881deca304e27f9cb20304c81731433954bec4f,"Everything is now OpenRA, not OpenRa",alzeih,2010-02-27 21:10:22+13:00,True,False,21,26919,26898,299,bleed,Actor.cs,11b926a42295e232676383c8909a0a1bcb0a101d,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645
1,8e5a5552c4fc078a44672294ed14570375037f78,Killed a bunch of logging statements,alzeih,2010-03-18 19:50:11+13:00,True,False,17,1,-16,10,bleed,Actor.cs,ec544489945ed0e895cf618583e2190f4e72f386,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645
2,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,"impose the hack on WALLs, not everything-else",Chris Forbes,2010-03-27 10:05:01+13:00,True,False,41,33,-8,6,bleed,Actor.cs,dd6f61a29f8cc3059231f3d22470c664c151d819,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645
3,13e6711a3b679a8522de76d2724c70798b3f40cb,further unhacking to localise extended damage ...,Chris Forbes,2010-03-27 10:11:24+13:00,True,False,27,27,0,4,bleed,Actor.cs,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645
4,d2ad90aa5b54e4bdb87b0f5215ba046bd0f2af2d,"saner planes, part 1",Chris Forbes,2010-03-27 00:23:24+13:00,True,False,2,54,52,4,bleed,Actor.cs,092c9bede0e15099e6583c532fb1c536f15419e7,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645


In [223]:
target_file = 'OpenRA.Game\\Game.cs'
hashes = list(df_merged[df_merged['fullpath'] == target_file]['hash'])

In [224]:
# Let's determine if a commit is related to an arbitrary file
def set_related_to_file(row):

    if row['fullpath'] == target_file or row['hash'] in hashes:
        row['is_related_to_file'] = 1
    else:
        row['is_related_to_file'] = 0

    return row

df_merged = df_merged.apply(set_related_to_file, axis=1)
df_merged.tail()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,...,filename_x,parents,fullpath,is_bug,project,path,filename_y,ext,lines,is_related_to_file
19488,9e92340ea7d16833731ed59c1a74c4e8b197024a,Rework chat line templates and logic\r\n\r\n- ...,Ivaylo Draganov,2021-06-15 17:32:21+03:00,True,False,363,330,-33,20,...,TextNotificationsDisplayWidget.cs,8416dc3f2dde1d1cbda590c7ddae974b82273257,OpenRA.Mods.Common\Widgets\TextNotificationsDi...,0,OpenRA.Mods.Common,Widgets,TextNotificationsDisplayWidget.cs,.cs,136,0
19489,01d47566cc10ff3fc0380cd3e669676614911f94,Add a transients panel with corresponding chro...,Ivaylo Draganov,2021-06-16 19:06:15+03:00,True,False,38,155,117,20,...,TextNotificationsDisplayWidget.cs,9e92340ea7d16833731ed59c1a74c4e8b197024a,OpenRA.Mods.Common\Widgets\TextNotificationsDi...,0,OpenRA.Mods.Common,Widgets,TextNotificationsDisplayWidget.cs,.cs,136,0
19490,3310f14deaa5572663efcbee384118ced5a6679d,Use mission notifications pool with appropriat...,Ivaylo Draganov,2021-11-05 20:21:44+02:00,True,False,8,13,5,9,...,TextNotificationsDisplayWidget.cs,a71da0a25a25977692a08e459bf427d91598e874,OpenRA.Mods.Common\Widgets\TextNotificationsDi...,0,OpenRA.Mods.Common,Widgets,TextNotificationsDisplayWidget.cs,.cs,136,0
19491,0f01df5474d19d88d921f94810d166d64fd7ecc7,Avoid string allocations in MiniYaml parsing.\...,RoosterDragon,2021-10-10 12:41:54+01:00,True,False,42,165,123,9,...,StreamExtsTests.cs,270c5665706f992736c74d37a463e99da8c9970d,OpenRA.Test\OpenRA.Game\StreamExtsTests.cs,0,OpenRA.Test,OpenRA.Game,StreamExtsTests.cs,.cs,73,0
19492,a537346580d8ceeeb4c1e7ed78ee94e8ef84f81c,Move selection hotkeys out of world interactio...,Ivaylo Draganov,2021-09-19 21:44:05+03:00,True,False,141,248,107,6,...,SelectionUtils.cs,e9cc89a33661a904fdb18c1cb1033481371ccaed,OpenRA.Mods.Common\Widgets\SelectionUtils.cs,0,OpenRA.Mods.Common,Widgets,SelectionUtils.cs,.cs,83,0


In [225]:
import random

def determine_time_spent(row):
    # For demonstration purposes, let's say each row took a random amount of time between 5 minutes and 8 hours. 
    # In the real world we'd get this from a time tracking system or ticketing system, but this random number generation should let us simulate graphing time spent on an area
    row['minutes'] = random.randrange(5, 480)
    return row

df_merged = df_merged.apply(determine_time_spent, axis=1)

df_merged.head()

,hash,message,author_name,author_date,in_main,is_merge,num_deletes,num_inserts,net_lines,num_files,...,parents,fullpath,is_bug,project,path,filename_y,ext,lines,is_related_to_file,minutes
0,7881deca304e27f9cb20304c81731433954bec4f,"Everything is now OpenRA, not OpenRa",alzeih,2010-02-27 21:10:22+13:00,True,False,21,26919,26898,299,...,11b926a42295e232676383c8909a0a1bcb0a101d,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645,1,321
1,8e5a5552c4fc078a44672294ed14570375037f78,Killed a bunch of logging statements,alzeih,2010-03-18 19:50:11+13:00,True,False,17,1,-16,10,...,ec544489945ed0e895cf618583e2190f4e72f386,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645,1,315
2,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,"impose the hack on WALLs, not everything-else",Chris Forbes,2010-03-27 10:05:01+13:00,True,False,41,33,-8,6,...,dd6f61a29f8cc3059231f3d22470c664c151d819,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645,0,127
3,13e6711a3b679a8522de76d2724c70798b3f40cb,further unhacking to localise extended damage ...,Chris Forbes,2010-03-27 10:11:24+13:00,True,False,27,27,0,4,...,faa8f54e7cd71ef3dcc4c526baa8b6639e1a49ed,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645,0,76
4,d2ad90aa5b54e4bdb87b0f5215ba046bd0f2af2d,"saner planes, part 1",Chris Forbes,2010-03-27 00:23:24+13:00,True,False,2,54,52,4,...,092c9bede0e15099e6583c532fb1c536f15419e7,OpenRA.Game\Actor.cs,0,OpenRA.Game,.,Actor.cs,.cs,645,0,16


In [226]:
from scipy import stats

df_grouped = df_merged.groupby(['fullpath', 'project', 'path', 'filename_y']).agg(
    avg_net=('net_lines','mean'),
    commits=('author_date', 'count'),
    lines=('lines','max'),
    ext=('ext','max'),
    avg_files=('num_files','mean'),
    first_modified=('author_date', 'min'),
    last_modified=('author_date','max'),
    sum_minutes=('minutes', 'sum'),
    avg_minutes=('minutes', 'mean'),
    pct_bug=('is_bug', 'mean'),
    num_bug=('is_bug', 'sum'),
    pct_related=('is_related_to_file', 'mean'),
    author_name=('author_name', lambda x:stats.mode(x)[0]),
    last_author=('author_name', lambda x:x.iloc[-1]),
    first_author=('author_name', lambda x:x.iloc[0])
)
df_grouped.head()

,,,,avg_net,commits,lines,ext,avg_files,first_modified,last_modified,sum_minutes,avg_minutes,pct_bug,num_bug,pct_related,author_name,last_author,first_author
fullpath,project,path,filename_y,,,,,,,,,,,,,,,
OpenRA.Game\Activities\Activity.cs,OpenRA.Game,Activities,Activity.cs,21.125000,48,291,.cs,230.708333,2014-12-14 17:06:29+01:00,2021-06-29 16:34:36+02:00,12180,253.750000,0.083333,4,0.229167,Oliver Brakmann,Andre Mohren,reaperrr
OpenRA.Game\Activities\CallFunc.cs,OpenRA.Game,Activities,CallFunc.cs,59.928571,14,33,.cs,687.285714,2014-12-14 17:06:29+01:00,2021-08-12 22:38:36+02:00,4661,332.928571,0.000000,0,0.642857,abcdefg30,Vapre,reaperrr
OpenRA.Game\Actor.cs,OpenRA.Game,.,Actor.cs,133.613953,215,645,.cs,80.260465,2010-02-27 21:10:22+13:00,2021-10-29 15:17:17+02:00,51598,239.990698,0.176744,38,0.158140,Paul Chote,Matthias Mailänder,alzeih
OpenRA.Game\CPos.cs,OpenRA.Game,.,CPos.cs,32.928571,42,150,.cs,282.642857,2012-06-20 23:22:27-05:00,2021-09-25 12:52:08+01:00,9725,231.547619,0.095238,4,0.333333,Paul Chote,RoosterDragon,James Dunne
OpenRA.Game\CVec.cs,OpenRA.Game,.,CVec.cs,64.678571,28,122,.cs,428.571429,2012-06-20 23:22:27-05:00,2021-06-29 16:34:36+02:00,7653,273.321429,0.142857,4,0.571429,Paul Chote,Andre Mohren,James Dunne


In [227]:
# Define common variables for plotly
from Theming import theme, template, theme_sequential

In [228]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="sum_minutes",
                   color=df_grouped.index.get_level_values(1),
                   title='Total Minutes Spent per File',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [229]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="avg_net",
                   color=df_grouped.index.get_level_values(1),
                   title='Average Net Lines',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [230]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="commits",
                   color=df_grouped.index.get_level_values(1),
                   title='# Commits Affecting Each File',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [231]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="pct_bug",
                   color=df_grouped.index.get_level_values(1),
                   title='% of Commits Dealing with Bugs (per file)',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [232]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="first_modified",
                   color=df_grouped.index.get_level_values(1),
                   title='First Modified Date',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [233]:
# Histogram of lines of code
fig = px.histogram(df_grouped,
                   x="last_modified",
                   color=df_grouped.index.get_level_values(1),
                   title='Last Modified Date',
                   labels={'lines': 'Lines of Code'},
                   template=template,
                   color_discrete_sequence=theme)
fig.show()

In [234]:
# Overall box plot for all code
fig = px.box(df_grouped,
             title='# Bugs by Project',
             x='num_bug',
             color=df_grouped.index.get_level_values(1),
             template=template,
             color_discrete_sequence=theme,
             #hover_data=['project','path','filename'],
             labels={
                 'project': 'Project',
                 'lines': 'Lines of Code',
             },
             points='suspectedoutliers')
fig.update_traces(quartilemethod='linear', jitter=1)
fig.show()

In [235]:
# Overall box plot for all code
fig = px.box(df_grouped,
             title='% Bugs by Project',
             x='pct_bug',
             color=df_grouped.index.get_level_values(1),
             template=template,
             color_discrete_sequence=theme,
             #hover_data=['project','path','filename'],
             labels={
                 'project': 'Project',
                 'lines': 'Lines of Code',
             },
             points='suspectedoutliers')
fig.update_traces(quartilemethod='linear', jitter=1)
fig.show()

In [236]:
# Overall box plot for all code
fig = px.box(df_grouped,
             title='Distribution of Lines of Code',
             x='avg_net',
             color=df_grouped.index.get_level_values(1),
             template=template,
             color_discrete_sequence=theme,
             #hover_data=['project','path','filename'],
             labels={
                 'project': 'Project',
                 'lines': 'Lines of Code',
             },
             points='suspectedoutliers')
fig.update_traces(quartilemethod='linear', jitter=1)
fig.show()

In [237]:
# Box plot for distribution by projects
fig = px.box(df_grouped,
             x='avg_net',
             y=df_grouped.index.get_level_values(1),
#             hover_data=['path','filename'],
             labels={
               'project': 'Project',
               'lines': 'Lines of Code',
             },
             title='Distribution of Lines of Code by Project',
             points='suspectedoutliers',
             template=template,
             color_discrete_sequence=theme,
             color=df_grouped.index.get_level_values(1))
fig.update_traces(quartilemethod='linear', jitter=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickangle=30, tickfont={'size': 10})
fig.show()

In [238]:
# Box plot for commits by projects
fig = px.box(df_grouped,
             x='commits',
             y=df_grouped.index.get_level_values(1),
#             hover_data=['path','filename'],
             labels={
               'project': 'Project',
               'lines': 'Lines of Code',
             },
             title='Commits by File by Project',
             points='suspectedoutliers',
             template=template,
             color_discrete_sequence=theme,
             color=df_grouped.index.get_level_values(1))
fig.update_traces(quartilemethod='linear', jitter=0.5)
fig.update_layout(showlegend=False)
fig.update_yaxes(tickangle=30, tickfont={'size': 10})
fig.show()

In [239]:
fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='sum_minutes',
                 title='Files by Minutes Spent',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [240]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='commits',
                 title='Files by Number of Commits',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [241]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='num_bug',
                 title='Files by # of Commits Related to Bug Fixes',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [242]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='pct_bug',
                 title='Files by % of Commits Related to Bug Fixes',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [243]:
fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='pct_related',
                 title='Files by % of Commits involving Target File',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [244]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='avg_files',
                 title='Files by Average # of other files modified',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_continuous_scale='balance',
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [245]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='last_modified',
                 title='Files by Date Last Modified',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.diverging.RdYlBu,                 
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [246]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='first_modified',
                 title='Files by Date Created',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.diverging.RdYlBu,
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [247]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='first_author',
                 title='File Creator',
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [248]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='last_author',
                 title='File Last Modified by',
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [249]:
# Cool color scales: matter, icefire, darkmint, picnic, temps, balance, tempo

# TODO: Using Graph Objects here would allow for a more dynamic hierarchy

fig = px.treemap(df_grouped,
                 path=[px.Constant('All Code'),df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='author_name',
                 title='File Ownership',
                 #hover_name='filename',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [250]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df_grouped,
                 path=['author_name', df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='author_name',
                 title='Files by Most Frequent Author',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 width=1024,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [251]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df_grouped,
                 path=['last_author', df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='last_author',
                 title='Files by Last Author',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 width=1024,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [252]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df_grouped,
                 path=['first_author', df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='first_author',
                 title='Files by First Author',
                 #hover_data=['fullpath'],
                 color_discrete_sequence=px.colors.qualitative.Antique,
                 template=template,
                 width=1024,
                 height=800,
                 values='lines')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [253]:
# Sunburst diagram. Same data as a treemap, but different presentation
fig = px.sunburst(df_grouped,
                 path=[df_grouped.index.get_level_values(1),df_grouped.index.get_level_values(2),df_grouped.index.get_level_values(3)],
                 color='lines',
                 title='Files size by commit frequency (size)',
                 #hover_data=['fullpath'],
                 color_continuous_scale='sunsetdark',
                 template=template,
                 width=1024,
                 height=800,
                 values='commits')
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()